# Waveframe v4.0 — Plug‑and‑Play Quickstart (Colab & local)

Run **Runtime → Run all**. This notebook will:
1) Install minimal dependencies
2) Clone the repo (if not already present in Colab)
3) Auto-discover CSVs in `Analysis/` and `Demos/Data/` (recursive)
4) Fall back to an upload dialog if none are found
5) Plot common observables when columns exist


In [ ]:
# 1) Install + import deps (safe to run multiple times)
!pip -q install numpy pandas matplotlib
import sys, platform, os, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print('Python:', sys.version)
print('Platform:', platform.platform())
print('pandas:', pd.__version__)
print('numpy:', np.__version__)
print('matplotlib:', plt.matplotlib.__version__)


In [ ]:
# 2) Ensure the repo is available (Colab)
REPO_URL = 'https://github.com/Wright-Shawn/Waveframe-v4.0.git'
REPO_DIR = '/content/Waveframe-v4.0'
if os.path.isdir('/content') and not os.path.isdir(REPO_DIR):
    print('Cloning repo into Colab workspace...')
    !git clone $REPO_URL
else:
    print('Repo already present or running locally.')

# Switch working dir if in Colab
if os.path.isdir(REPO_DIR):
    os.chdir(REPO_DIR)
print('CWD:', os.getcwd())


In [ ]:
# 3) Discover CSVs
CANDIDATE_DIRS = ['Analysis', 'analysis', 'Demos/Data']
csvs = []
for rel in CANDIDATE_DIRS:
    root = os.path.abspath(rel)
    found = sorted(glob.glob(os.path.join(root, '**', '*.csv'), recursive=True))
    if found:
        print(f'Found {len(found)} CSV(s) under {rel}')
        csvs.extend(found)

print('\nAll discovered CSVs:')
for i, p in enumerate(csvs):
    print(f'  [{i}] {p}')

if not csvs:
    print('\nNo CSVs found. Use the upload dialog below to add one.')
    try:
        from google.colab import files
        uploaded = files.upload()  # opens file picker UI
        os.makedirs('Uploaded', exist_ok=True)
        for name, data in uploaded.items():
            with open(os.path.join('Uploaded', name), 'wb') as f:
                f.write(data)
            csvs.append(os.path.abspath(os.path.join('Uploaded', name)))
        print('Uploaded CSVs registered:')
        for i, p in enumerate(csvs):
            print(f'  [{i}] {p}')
    except Exception as e:
        raise FileNotFoundError('No CSVs found and upload is unavailable in this environment.')

# Choose which CSV to load
CSV_INDEX = 0  # change if you have multiple CSVs and want a different one
csv_path = csvs[CSV_INDEX]
print('\nUsing CSV ->', csv_path)
df = pd.read_csv(csv_path)
print('Columns:', list(df.columns))
df.head()


In [ ]:
# 4) Detect columns and plot
def pick(first_match_from, columns):
    for name in first_match_from:
        if name in columns:
            return name
    return None

Z = pick(['z','redshift','Z'], df.columns)
E = pick(['E','E_z','E(z)'], df.columns)
DA = pick(['D_A','D_A(z)','DA','DA_z','angular_diameter_distance'], df.columns)
H = pick(['H','H_z','H(z)','Hz'], df.columns)
G = pick(['growth','f_sigma8','fσ8','f_s8','fs8'], df.columns)

print('Detected ->', {'z':Z, 'E':E, 'D_A':DA, 'H':H, 'growth':G})

def maybe_plot(xname, yname, title):
    if xname and yname:
        plt.figure()
        plt.plot(df[xname], df[yname])
        plt.xlabel(xname); plt.ylabel(yname); plt.title(title)
        plt.grid(True)
        plt.show()
        return True
    else:
        print(f'Skipping {title}: missing columns')
        return False

maybe_plot(Z, E, 'E(z)')
maybe_plot(Z, DA, 'D_A(z)')
maybe_plot(Z, H, 'H(z)')
maybe_plot(Z, G, 'Growth (proxy)')


### Optional: Save figures
Uncomment and run the cell below to save all open figures into `Figures/auto/`.

In [ ]:
# SAVE = False
# if SAVE:
#     out_dir = os.path.abspath(os.path.join('Figures', 'auto'))
#     os.makedirs(out_dir, exist_ok=True)
#     for i, num in enumerate(plt.get_fignums(), start=1):
#         fig = plt.figure(num)
#         fig.savefig(os.path.join(out_dir, f'plot_{i:02d}.png'), dpi=150, bbox_inches='tight')
#     print('Saved to:', out_dir)
